In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6220/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_6220/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_48_5,0.892954,0.658642,0.543344,0.789399,0.028510,0.202645,0.216697,0.028083,0.076725,0.168850,0.857978,0.170114,73.114987,168.989055,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_48_9,0.892924,0.658502,0.541897,0.786474,0.028518,0.202728,0.217383,0.028474,0.076748,0.168873,0.857939,0.170138,73.114444,168.988511,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_17_9,0.896324,0.658266,0.580749,0.857432,0.027612,0.202868,0.198947,0.039548,0.076785,0.166170,0.862450,0.167415,73.178978,169.053046,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_48_2,0.893219,0.658110,0.544121,0.801969,0.028440,0.202961,0.216328,0.026407,0.076801,0.168640,0.858330,0.169904,73.119948,168.994016,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_32_2,0.897740,0.657786,0.566304,0.837932,0.027235,0.203153,0.205801,0.028648,0.076444,0.165031,0.864329,0.166267,73.206483,169.080551,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,model_4_10_4,0.893887,-5.788417,0.446347,0.816064,0.028262,4.029897,0.262724,0.079023,0.060415,0.168112,0.615699,0.169371,201.132506,482.944160,"Hidden Size=[24], regularizer=0.02, learning_r..."
791,model_4_11_0,0.935622,-6.234023,0.505575,0.935737,0.017146,4.294429,0.234619,0.012832,0.055322,0.130943,0.766848,0.131924,202.131977,483.943631,"Hidden Size=[24], regularizer=0.02, learning_r..."
792,model_4_10_8,0.947380,-6.324781,0.290565,0.944697,0.014015,4.348307,0.336648,0.023759,0.054489,0.118383,0.809430,0.119270,202.535319,484.346972,"Hidden Size=[24], regularizer=0.02, learning_r..."
793,model_4_9_9,0.902458,-7.079136,0.564488,0.641271,0.025979,4.796124,0.206663,0.085162,0.065670,0.161179,0.646741,0.162387,201.300953,483.112607,"Hidden Size=[24], regularizer=0.02, learning_r..."
